In [11]:
import os
import re
import shutil
import string
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, losses

e:\Tensorflow-Tutorial\venv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [6]:
import pandas as pd
from ftfy import fix_text

def fix_vi_text(text):
    try:
        text = text.encode("latin1").decode("utf-8")
    except:
        pass
    text = fix_text(text)
    return text

df = pd.read_csv("data/VI_IMDB.csv", encoding="latin1")
print(df["vi_review"].iloc[0])

df["vi_review"] = df["vi_review"].astype(str).apply(fix_vi_text)
df.to_csv("data/VI_IMDB_fixed.csv", index=False, encoding="utf-8")

# vefifying the fix
df = pd.read_csv("data/VI_IMDB_fixed.csv", encoding="utf-8")
print(df["vi_review"].iloc[0])

Má»t trong nhá»¯ng ngÆ°á»i ÄÃ¡nh giÃ¡ khÃ¡c ÄÃ£ Äá» cáº­p ráº±ng sau khi xem chá» 1 táº­p Oz, báº¡n sáº½ bá» cuá»n hÃºt. Há» nÃ³i ÄÃºng, vÃ¬ ÄÃ¢y chÃ­nh xÃ¡c lÃ  nhá»¯ng gÃ¬ ÄÃ£ xáº£y ra vá»i tÃ´i.<br /><br />Äiá»u Äáº§u tiÃªn khiáº¿n tÃ´i áº¥n tÆ°á»£ng vá» Oz lÃ  sá»± tÃ n báº¡o vÃ  nhá»¯ng cáº£nh báº¡o lá»±c khÃ´ng há» nao nÃºng, thá» hiá»n ngay tá»« chá»¯ ÄI. Tin tÃ´i Äi, ÄÃ¢y khÃ´ng pháº£i lÃ  má»t chÆ°Æ¡ng trÃ¬nh dÃ nh cho nhá»¯ng ngÆ°á»i yáº¿u tim hay nhÃºt nhÃ¡t. ChÆ°Æ¡ng trÃ¬nh nÃ y khÃ´ng cÃ³ cÃº Äáº¥m nÃ o liÃªn quan Äáº¿n ma tÃºy, tÃ¬nh dá»¥c hay báº¡o lá»±c. NÃ³ lÃ  háº¡ng náº·ng, theo cÃ¡ch sá»­ dá»¥ng cá» Äiá»n cá»§a tá»« nÃ y.<br /><br />NÃ³ ÄÆ°á»£c gá»i lÃ  OZ vÃ¬ ÄÃ³ lÃ  biá»t danh ÄÆ°á»£c Äáº·t cho Tráº¡i giam NhÃ  nÆ°á»c An ninh Tá»i Äa Oswald. NÃ³ táº­p trung chá»§ yáº¿u vÃ o ThÃ nh phá» Ngá»c lá»¥c báº£o, má»t khu vá»±c thá»­ nghiá»m cá»§a nhÃ  tÃ¹ nÆ¡i táº¥t cáº£ cÃ¡c phÃ²ng giam Äá»u cÃ³ máº·t trÆ°á»c báº±ng kÃ­nh vÃ

In [22]:
from pathlib import Path

import pandas as pd
import tensorflow as tf
import re
import string

from sklearn.model_selection import train_test_split



# Paths and parameters

csv_path = Path("data/VI_IMDB_fixed.csv")

output_root = Path("data/vi_imdb_split")

test_size = 0.2

random_state = 42



def custom_standardization(input_data: tf.Tensor) -> tf.Tensor:
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )



# Load data and pick the Vietnamese review column

df = pd.read_csv(csv_path, encoding="utf-8")

review_cols = [c for c in df.columns if "vi_review" in c]

if not review_cols:

    raise ValueError("No column containing 'vi_review' found in the CSV")

text_col = review_cols[0]

if "sentiment" not in df.columns:

    raise ValueError("Column 'sentiment' not found in the CSV")



# Keep only needed columns and drop missing rows

df = df[[text_col, "sentiment"]].dropna()



# Train/test split (stratified so class balance is preserved)

train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    random_state=random_state,
    stratify=df["sentiment"],
)



def write_split(split_df: pd.DataFrame, split_name: str) -> Path:
    # Standardize text for this split and write one review per txt under pos/neg
    standardized = custom_standardization(
        tf.constant(split_df[text_col].astype(str).values)
    ).numpy()
    split_df = split_df.copy()
    split_df[text_col] = [s.decode("utf-8", errors="ignore") for s in standardized]

    split_dir = output_root / split_name
    for label_value, group in split_df.groupby(split_df["sentiment"].str.lower().str.strip()):

        if label_value not in {"positive", "negative"}:

            continue

        label_dir = split_dir / ("pos" if label_value == "positive" else "neg")

        label_dir.mkdir(parents=True, exist_ok=True)

        for idx, text in enumerate(group[text_col], start=1):

            file_path = label_dir / f"{split_name}_{label_value}_{idx}.txt"

            file_path.write_text(str(text).strip(), encoding="utf-8")
    return split_dir



train_dir = write_split(train_df, "train")

test_dir = write_split(test_df, "test")



print(f"Train folder: {train_dir.resolve()}")

print(f"Test folder:  {test_dir.resolve()}")

print(f"Wrote dataset to {output_root.resolve()}")

Train folder: E:\Tensorflow-Tutorial\Vietnamese Text Classification\data\vi_imdb_split\train
Test folder:  E:\Tensorflow-Tutorial\Vietnamese Text Classification\data\vi_imdb_split\test
Wrote dataset to E:\Tensorflow-Tutorial\Vietnamese Text Classification\data\vi_imdb_split


In [23]:
# Datasets
def setup_dataset(train_dir, test_dir):
    batch_size = 32
    seed = 42
    raw_train_ds = tf.keras.utils.text_dataset_from_directory(
        train_dir,
        batch_size=batch_size,
        validation_split=0.2,
        subset="training",
        seed=seed,
    )
    amount_batches = tf.data.experimental.cardinality(raw_train_ds)
    print(f'Number of batches in the training dataset: {amount_batches}')

    raw_val_ds = tf.keras.utils.text_dataset_from_directory(
        train_dir,
        batch_size=batch_size,
        validation_split=0.2,
        subset="validation",
        seed=seed,
    )
    amount_batches = tf.data.experimental.cardinality(raw_val_ds)
    print(f'Number of batches in the validation dataset: {amount_batches}')

    raw_test_ds = tf.keras.utils.text_dataset_from_directory(
        test_dir,
        batch_size=batch_size,
    )
    amount_batches = tf.data.experimental.cardinality(raw_test_ds)
    print(f'Number of batches in the test dataset: {amount_batches}')

    return raw_train_ds, raw_val_ds, raw_test_ds

raw_train_ds, raw_val_ds, raw_test_ds = setup_dataset(train_dir, test_dir)


Found 40000 files belonging to 2 classes.
Using 32000 files for training.
Number of batches in the training dataset: 1000
Found 40000 files belonging to 2 classes.
Using 8000 files for validation.
Number of batches in the validation dataset: 250
Found 10000 files belonging to 2 classes.
Number of batches in the test dataset: 313


In [24]:
text_batch, label_batch = next(iter(raw_train_ds))
print("Length of text batch: ", len(text_batch))

Length of text batch:  32


In [ ]:
from sentence_transformers import SentenceTransformer

# Use SBERT for vectorization with UTF-8 decoding to avoid ASCII decode issues
sbert = SentenceTransformer("keepitreal/vietnamese-sbert")
embedding_dim = sbert.get_sentence_embedding_dimension()


def encode_text_batch(text_batch: tf.Tensor) -> tf.Tensor:
    # Decode bytes -> utf-8 (ignore errors) then encode with SBERT
    sentences = [s.decode("utf-8", errors="ignore") for s in text_batch.numpy().tolist()]
    embeddings = sbert.encode(sentences, convert_to_numpy=True, show_progress_bar=False)
    return embeddings.astype("float32")


def vectorize_dataset(dataset: tf.data.Dataset) -> tf.data.Dataset:
    def _map(text, label):
        embeddings = tf.py_function(func=encode_text_batch, inp=[text], Tout=tf.float32)
        embeddings.set_shape((None, embedding_dim))  # batch, embedding_dim
        label = tf.cast(label, tf.float32)
        return embeddings, label

    return dataset.map(_map, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)


vectorized_train_ds = vectorize_dataset(raw_train_ds)
vectorized_val_ds = vectorize_dataset(raw_val_ds)
vectorized_test_ds = vectorize_dataset(raw_test_ds)

In [28]:
# MLP model on SBERT embeddings (shape uses embedding_dim)
mlp_model = tf.keras.Sequential(
    [
        layers.Input(shape=(embedding_dim,)),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ]
)

mlp_model.compile(
    loss=losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")],
)

print(mlp_model.summary())

# history = mlp_model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=10,
#     verbose=1,
# )

# eval_results = mlp_model.evaluate(test_ds, verbose=1)
# print("Test results (loss, accuracy, auc):", eval_results)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 256)            │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 213,377 (833.50 KB)

 Trainable params: 213,377 (833.50 KB)

 Non-trainable params: 0 (0.00 B)

None
